<a href="https://colab.research.google.com/github/jojoconverteo/Evaneos/blob/main/Evaneos_Churn_(clustering).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://www.offremedia.com/sites/default/files/vignette/article/converteo-logo.png)

# **Churn : Analyse non supervisée**


**Contexte** : On cherche à savoir si un client mobile est à risque de désengagement


-------------------------------

**Dictionnaire :**

Churn (Cible) :
-  1 if customer cancelled service, 0 if not

AccountWeeks : 
- number of weeks customer has had active account


DataPlan : 
- 1 if customer has data plan, 0 if not

DataUsage : 
 - gigabytes of monthly data usage


CustServCalls : 
- number of calls into customer service


DayMins :
- average daytime minutes per month


DayCalls : 
- average number of daytime calls


MonthlyCharge :
- average monthly bill


OverageFee :
- largest overage fee in last 12 months

ContractRenewal :
- 1 if customer recently renewed contract, 0 if not


RoamMins : 
- average number of roaming minutes [LE ROAMING, C'EST QUOI ?](https://www.sfrbusiness.fr/room/communications-unifiees/roaming-c-est-quoi.html)

In [1]:
#@title


!pip install plotly --upgrade
!pip install -U pandas_profiling

from pandas_profiling import ProfileReport
from google.colab import drive
import os 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import typing
from typing import List
import numpy as np


import warnings
warnings.filterwarnings('ignore')

def func_create_noise(df_train_data: pd.DataFrame, coloumns_cat_2_category: List) -> pd.DataFrame:
  """
  Fonction qui permet de creer du bruit


  Parameters:
  ----------------------------
    df_train_data: pd.DataFrame 
    Dataframe d'entree 


  Return:
  -----------------------------
    df_train_data_suffle: pd.DataFrame
    Dataframe modifier  

  """
  dict_create_noise_columns_cat = {1 : 'Yes', 0 : 'No'}
  df_train_data_suffle = df_train_data.sample(frac=1)

  for col in columns_cat_2_category: 
    df_train_data_suffle[col] = df_train_data_suffle[col].apply(lambda x: dict_create_noise_columns_cat[x])

  return df_train_data_suffle




drive.mount('/content/drive')
sep = os.sep
str_path_to_file = f"/content/drive/My Drive/Cours Data/Classification/Data/telecom_churn.csv"

df_train_data_suffle = pd.read_csv(str_path_to_file, encoding='ascii')

columns_cat_2_category=['Churn', 'ContractRenewal', 'DataPlan']
df_train_data_suffle = func_create_noise(df_train_data_suffle, columns_cat_2_category)
df_train_data_suffle = df_train_data_suffle.sample(frac=1)

df_train_data_suffle_cat = df_train_data_suffle.select_dtypes(include=object)
df_train_data_suffle_num = df_train_data_suffle.select_dtypes(exclude=object)

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.12.0)
Requirement already up-to-date: pandas_profiling in /usr/local/lib/python3.6/dist-packages (2.9.0)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


-----------------
 
### K-means
 







In [2]:
pd.get_dummies(df_train_data_suffle_cat)

Churn_No  Churn_Yes  ...  DataPlan_No  DataPlan_Yes
2793         1          0  ...            0             1
2109         1          0  ...            0             1
705          1          0  ...            1             0
1913         1          0  ...            1             0
1780         1          0  ...            1             0
...        ...        ...  ...          ...           ...
95           1          0  ...            1             0
1717         1          0  ...            1             0
759          1          0  ...            0             1
2880         1          0  ...            1             0
2472         0          1  ...            1             0

[3333 rows x 6 columns]

-------------------------------------
One-hot-encoding amelioré

In [3]:
#@title

df_train_data_one_hot_encoding = pd.get_dummies(df_train_data_suffle_cat, drop_first=True)
df_train_data_one_hot_encoding

Churn_Yes  ContractRenewal_Yes  DataPlan_Yes
2793          0                    1             1
2109          0                    1             1
705           0                    1             0
1913          0                    1             0
1780          0                    1             0
...         ...                  ...           ...
95            0                    1             0
1717          0                    1             0
759           0                    1             1
2880          0                    1             0
2472          1                    1             0

[3333 rows x 3 columns]

In [4]:
#@title

df_train_data_kmeans = pd.concat([df_train_data_suffle_num, df_train_data_one_hot_encoding], axis=1)
df_train_data_kmeans

AccountWeeks  DataUsage  ...  ContractRenewal_Yes  DataPlan_Yes
2793           107       3.89  ...                    1             1
2109           114       3.40  ...                    1             1
705             99       0.00  ...                    1             0
1913            65       0.28  ...                    1             0
1780            68       0.00  ...                    1             0
...            ...        ...  ...                  ...           ...
95             104       0.30  ...                    1             0
1717            70       0.00  ...                    1             0
759            136       2.57  ...                    1             1
2880           107       0.00  ...                    1             0
2472           129       0.23  ...                    1             0

[3333 rows x 11 columns]

In [5]:
#@title

from sklearn.preprocessing import StandardScaler

df_train_data_kmeans = pd.concat([df_train_data_suffle_num, df_train_data_one_hot_encoding], axis=1)
sk_scaler = StandardScaler()
sk_scaler.fit(df_train_data_kmeans)
array_train_data_kmeans = sk_scaler.transform(df_train_data_kmeans)

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
 
 
inertia = []
for n_clusters in range(2, 14):
   kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(array_train_data_kmeans)
   inertia.append(kmeans.inertia_ / n_clusters)
 
inertias = pd.DataFrame({'n_clusters': range(2, 14), 'inertia': inertia})
 
fig = px.line(inertias, x="n_clusters", y="inertia", title='Inertie (somme des variances des clusters)')
fig.show()



In [6]:
#@title
kmeans = KMeans(n_clusters=3, random_state=0).fit(array_train_data_kmeans)

df_train_data_kmeans['Clusteur'] = kmeans.labels_

In [7]:
#@title

import plotly.graph_objects as go

list_col_num = [w for w in df_train_data_suffle_num.columns]
list_col_little = ['DataUsage', 'CustServCalls', 'OverageFee', 'RoamMins']
list_col_num_big = [w for w in list_col_num if w not in list_col_little]
list_col_cat = [w for w in df_train_data_kmeans.columns if w not in list_col_num and w != 'Clusteur']

from typing import List

def return_viz_kmeans_compare(df_train_data_kmeans: pd.DataFrame, list_:List, *args) -> None:
  fig = go.Figure(data=[
      go.Bar(name='Clusteur_0', x=list_, y=df_train_data_kmeans.query('Clusteur == 0')[list_].mean().to_list()),
      go.Bar(name='Clusteur_1', x=list_, y=df_train_data_kmeans.query('Clusteur == 1')[list_].mean().to_list()),
      go.Bar(name='Clusteur_2', x=list_, y=df_train_data_kmeans.query('Clusteur == 2')[list_].mean().to_list())
  ])

  fig.update_layout(barmode='group')
  fig.show()



list_clusteur = ['Clusteur_0', 'Clusteur_1', 'Clusteur_3']
return_viz_kmeans_compare(df_train_data_kmeans , list_col_num_big)
return_viz_kmeans_compare(df_train_data_kmeans , list_col_little)
return_viz_kmeans_compare(df_train_data_kmeans , list_col_cat)